# Лекция 7 "Transformers"

- Monads as effects
- Reader & State
- MaybeT & Transformers
  - Проблемы композиции монад
  - MaybeIO
  - MaybeT
  - Общий вид transformers
  - Пример для __`MaybeT`__
- Reader
  - Environment + IO
  - Приколы ReaderT
- Transformers table
- Monad composing
- MonadThrow, MonadCatch, MonadError
- Дополнительно*


## Monads as effects

| __Monad__	| __Effect__                                        |
| :-------: | :------------------------------------------------ |
| Maybe	    | Computation can fail.                             |
| Either    | Computation can fail with annotated error.        |
| []        | Computation has multiple values.                  |
| Writer    | Computation has monoidal accumulator.             |
| Reader    | Computation has access to some immutable context. |
| State     | Computation has access to some mutable state.     |
| IO        | Computation can perform I/O actions.              |

## Reader & State

Хотим композицию монад __`Reader`__ & __`State`__.  
Проблемы: 
- Reader has ask but doesn't have put
- State has put but doesn't have ask

Одно из решений - это использовать монаду __`RWS`__:

```
foo :: RWS Int [Int] () Int
foo i = do
    baseCounter <- ask
    let newCounter = baseCounter + i
    put [baseCounter, newCounter]
    return newCounter
```

Но возникает бесполезная монада __`Writer`__, что нам не нравится.  
Поэтому придумали другой подход __Transformers__ (_`ReaderT`_ | _`StateT`_):

```
foo :: Int -> ReaderT Int (State [Int]) Int  -- or StateT [Int] (Reader Int) Int
foo i = do
    baseCounter <- ask
    let newCounter = baseCounter + i
    put [baseCounter, newCounter]
    return newCounter
```


## MaybeT & Transformers

### Проблемы композиции монад

If __f__ is a _Functor_ and __g__ is a _Functor_ then composition of __f__ and __g__ is also a _Functor_. Same for _Applicative, Alternative, Foldable, Traversable_.

__Haskell:__

```
instance (Functor     f, Functor     g) => Functor     (Compose f g)
instance (Foldable    f, Foldable    g) => Foldable    (Compose f g)
instance (Traversable f, Traversable g) => Traversable (Compose f g)
instance (Applicative f, Applicative g) => Applicative (Compose f g)
instance (Alternative f, Applicative g) => Alternative (Compose f g)
```

__!!! Сomposition of two monads can't be a Monad automatically__. Поэтому мы должны описывать все композиции монад по-отдельности.


### MaybeIO

```
newtype MaybeIO a = MaybeIO { runMaybeIO :: IO (Maybe a) }

instance Monad MaybeIO where
    return x = MaybeIO (return (Just x))
    MaybeIO action >>= f = MaybeIO $ do
        result <- action
        case result of
            Nothing -> return Nothing
            Just x  -> runMaybeIO (f x)
```

Теперь можно исполнить такой код:

```
result <- runMaybeIO $ do
    c1 <- MaybeIO $ tryConnect "host1"
    c2 <- MaybeIO $ tryConnect "host2"
    ...
```

__Но__ если мы захотим что-то вывести, то возникнут проблемы с типами. Поэтому напишем трансформер:

```
transformIO2MaybeIO :: IO a -> MaybeIO a
transformIO2MaybeIO action = MaybeIO $ do
    result <- action
    return (Just result)
```

Теперь можно спокойно запустить этот код:

```
result <- runMaybeIO $ do
  c1 <- MaybeIO $ tryConnect "host1"
  transformIO2MaybeIO $ print "Hello"
  c2 <- MaybeIO $ tryConnect "host2"
  ...
```

### MaybeT

Монада для использования композиции с __`Maybe`__:

```
newtype MaybeT m a = MaybeT 
    { runMaybeT :: m (Maybe a) }

instance Monad m => Monad (MaybeT m) where
    return :: a -> MaybeT m a
    return x = MaybeT (return (Just x))

    (>>=) :: MaybeT m a -> (a -> MaybeT m b) -> MaybeT m b
    MaybeT action >>= f = MaybeT $ do
        result <- action
        case result of
            Nothing -> return Nothing
            Just x  -> runMaybeT (f x)

transformToMaybeT :: Functor m => m a -> MaybeT m a
transformToMaybeT = MaybeT . fmap Just
```

### Общий вид transformers

Вместо того, чтобы описывать трансформеры для каждой конкретной раелизации:

```
transformIO2MaybeIO :: IO a -> MaybeIO a
transformIO2MaybeIO action = MaybeIO $ do
    result <- action
    return (Just result)
```

Люди придумали класс __`MonadTrans`__, который обобщает эту операцию и принимает на вход трансформер в монаду:

```
class MonadTrans t where    -- t :: (* -> *) -> * -> *
    lift :: Monad m => m a -> t m a
    
    {-# LAWS

        1. lift . return  ≡ return
        2. lift (m >>= f) ≡ lift m >>= (lift . f)

    #-}
```

И теперь наша для трансформирования одной монады в __`MaybeT`__ и другие достаточно описать такие трансформеры:

```
transformToMaybeT  :: Monad m => m a -> MaybeT    m a
transformToEitherT :: Monad m => m a -> EitherT l m a

instance MonadTrans MaybeT where
    lift :: Monad m => m a -> MaybeT m a
    lift = transformToMaybeT
```

### Пример для __`MaybeT`__

Рассмотрим пример обработки email-а:

```
emailIsValid :: String -> Bool
emailIsValid email = '@' `elem` email

askEmail :: MaybeT IO String
askEmail = do
    lift $ putStrLn "Input your email, please:"
    email <- lift getLine
    guard $ emailIsValid email
    return email

main :: IO ()
main = do
    Just email <- runMaybeT $ untilSuccess askEmail
    putStrLn $ "OK, your email is " ++ email

untilSuccess :: Alternative f => f a -> f a
untilSuccess = foldr (<|>) empty . repeat

-- Defined in Control.Monad.Trans.Maybe
instance (Functor m, Monad m) => Alternative (MaybeT m) where
    empty = MaybeT (return Nothing)
    x <|> y = MaybeT $ maybe (runMaybeT y) pure $ runMaybeT x
```

## Reader

### Environment + IO

IO безопаснее всего использовать с Reader. Например, работу с сервером или другим ресурсом, где важно хранить пароли, логи, юзернеймы ...

Рассмотрим пример с __`ReaderT`__:

```
newtype LoggerName = LoggerName { getLoggerName :: Text }

newtype ReaderT r m a = ReaderT { runReaderT :: r -> m a }

type LoggerIO a = ReaderT LoggerName IO a

logMessage :: Text -> LoggerIO ()

readFileWithLog :: FilePath -> LoggerIO Text
readFileWithLog path = do
    logMessage $ "Reading file: " <> T.pack (show path)
    lift $ readFile path

writeFileWithLog :: FilePath -> Text -> LoggerIO ()
writeFileWithLog path content = do
    logMessage $ "Writing to file: " <> T.pack (show path)
    lift $ writeFile path content

prettifyFileContent :: FilePath -> LoggerIO ()
prettifyFileContent path = do
    content <- readFileWithLog path
    writeFileWithLog path (format content)

main :: IO ()
main = runReaderT (prettifyFileContent "foo.txt") (LoggerName "Application") 
```

### Приколы ReaderT

Монада __`Reader`__ определена через монады __`ReaderT`__ и __`Identity`__:

```
type Reader r a = ReaderT r Identity a
 ```

## Transformers table

| __Base monad__ | __Transformer__ | __Original type__ | __Combined type__ |
| :------------: | :-------------- | :---------------- | :---------------- |
| Maybe          | MaybeT          | Maybe a	         | m (Maybe a)       |
| Either         | EitherT	       | Either a b	       | m (Either a b)    |
| Writer         | WriterT	       | (a, w)	           | m (a, w)          |
| Reader         | ReaderT	       | r -> a            | r -> m a          |
| State          | StateT	         | s -> (a, s)       | s -> m (a, s)     |
| Cont           | ContT	         | (a -> r) -> r	   | (a -> m r) -> m r |

Для __`IO`__ не существует трансформера, т. е. мы можем завернуть __`IO`__ в другую монаду, но вот любую другую монаду завернуть в __`IO`__ нельзя!!!



## Monad composing

Если мы захотим собрать композицию из некоторого числа монад с помощью __`MonadTrans`__, то нам надо будет прописывать уйму lift-ов.  
Чтобы этим не заниматься, люди создали класс __`MonadReader`__.  

Раньше было так (пришлось ручками явно прописать _`lift ask`_):

```
foo :: Int -> StateT [Int] (Reader Int) Int
foo i = do
    baseCounter <- lift ask
    let newCounter = baseCounter + i
    put [baseCounter, newCounter]
    return newCounter
```

Написали MonadReader:

```
class Monad m => MonadReader r m | m -> r where
    ask    :: m r
    local  :: (r -> r) -> m a -> m a
    reader :: (r -> a) -> m a

instance MonadReader r m => MonadReader r (StateT s m) where
    ask    = lift ask
    local  = mapStateT . local
    reader = lift . reader
```

Код превратился вот в это:

```
foo :: Int -> StateT [Int] (Reader Int) Int
foo i = do
    baseCounter <- ask
    let newCounter = baseCounter + i
    put [baseCounter, newCounter]
    return newCounter
```

То есть нам теперь не важно, что там внутри монады StateT. Мы просто пользуемся её функционалом. 


## MonadThrow, MonadCatch, MonadError

Существует монада для прокидывания ошибок:

```
class Monad m => MonadThrow m where
    throwM :: Exception e => e -> m a

instance MonadThrow Maybe where
    throwM _ = Nothing

instance MonadThrow IO where
    throwM = Control.Exception.throwIO

instance MonadThrow m => MonadThrow (StateT s m) where
    throwM = lift . throwM
```

Для отлавливания исключений:

```
class MonadThrow m => MonadCatch m where
    catch :: Exception e => m a -> (e -> m a) -> m a

instance MonadCatch IO where
    catch = Control.Exception.catch
```

Ну и монады для тех, кто хочет падать:

```
class (Monad m) => MonadError e m | m -> e where
    throwError :: e -> m a
    catchError :: m a -> (e -> m a) -> m a
```

Лучше использовать __`MonadThrow`__ в обычной жизни. 

## Дополнительно*

__mtl-style transformers__

* __`transformers`__ package:  
Has only MonadTrans type class and classic monad types (ReaderT, StateT, etc.)

* __`mtl`__ package:  
Reexports transformers. For each monad _`SomeT`_ adds _`MultiParamTypeClass`_ __`MonadSome`__ with _`FunctionalDependencies`_.

Поэтому юзайте __`mtl`__.

В этом коде прийдется все прописывать ручками:

```
-- Complex type for which we need to write all instances manually :(

newtype M a = M (Environment -> MyState -> IO (a, MyState))
```

А вот при помощи __`mtl`__ всё это может сделать __компилятор__:

```
{-# LANGUAGE FlexibleContexts           #-}
{-# LANGUAGE GeneralizedNewtypeDeriving #-}
{-# LANGUAGE MultiParamTypeClasses      #-}

-- Move all dirty work to compiler

newtype M a = M (ReaderT Environment (StateT MyState IO) a)
  deriving (Functor, Applicative, Monad, MonadIO, 
            MonadState MyState, MonadReader Environment)
```